In [169]:
# Importing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import numpy as np
from pycaret.regression import *
pd.set_option("display.max_columns", None)


df = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv"
)

crime_rate = pd.read_csv("./crime_rate.csv")

In [170]:
# Add crime rates from nearest city
df = pd.merge(df, crime_rate, on="zipcode", how="left")

In [171]:
df

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,property_crime_rate_closest_city,violent_crime_rate_closest_city,school_district
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,1732.6,89.4,Tahoma School District
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,5407.5,389.7,Federal Way School District
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,5522.0,598.7,Seattle Public Schools
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,5407.5,389.7,Federal Way School District
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,5522.0,598.7,Seattle Public Schools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,20141113T000000,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000.0,2221.1,60.0,Seattle Public Schools
19996,9578500790,20141111T000000,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950.0,5407.5,389.7,Federal Way School District
19997,7202350480,20140930T000000,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000.0,2966.5,105.7,Lake Washington School District
19998,1723049033,20140620T000000,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000.0,16583.8,750.6,Tukwila School District


In [172]:
def data_transform(df):

    df = pd.get_dummies(df, columns=["school_district"], drop_first=True)
    # Replace spaces with underscores in column names to avoid issues with LightGBM
    df.columns = df.columns.str.replace(" ", "_", regex=False)

    # Convert 'date' to datetime and extract year and month

    df["date"] = pd.to_datetime(df["date"])

    df["year_of_sale"] = df["date"].dt.year
    df["month_of_sale"] = df["date"].dt.month


    df.drop("date", axis=1, inplace=True)  # Drop the original date column


    # Add age of house


    df["age"] = 2015 - df["yr_built"]


    # Add binary has basement column


    df["has_basement"] = (df["sqft_basement"] > 0).astype(int)

    return df

In [173]:
df = data_transform(df)

In [174]:
df

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,property_crime_rate_closest_city,violent_crime_rate_closest_city,school_district_Bellevue_School_District,school_district_Enumclaw_School_District,school_district_Federal_Way_School_District,school_district_Highline_School_District,school_district_Issaquah_School_District,school_district_Kent_School_District,school_district_Lake_Washington_School_District,school_district_Mercer_Island_School_District,school_district_Northshore_School_District,school_district_Renton_School_District,school_district_Riverview_School_District,school_district_Seattle_Public_Schools,school_district_Shoreline_School_District,school_district_Snoqualmie_Valley_School_District,school_district_Tahoma_School_District,school_district_Tukwila_School_District,school_district_Vashon_Island_School_District,school_district_White_River_School_District,year_of_sale,month_of_sale,age,has_basement
0,1565930130,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,1732.6,89.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2014,11,8,1
1,3279000420,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,5407.5,389.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,1,36,1
2,194000575,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,5522.0,598.7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2014,10,101,0
3,2115510160,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,5407.5,389.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2014,12,30,0
4,7522500005,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,5522.0,598.7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2014,8,68,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000.0,2221.1,60.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2014,11,68,0
19996,9578500790,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950.0,5407.5,389.7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2014,11,1,0
19997,7202350480,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000.0,2966.5,105.7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2014,9,11,0
19998,1723049033,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000.0,16583.8,750.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2014,6,52,0


In [175]:
df.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price',
       'property_crime_rate_closest_city', 'violent_crime_rate_closest_city',
       'school_district_Bellevue_School_District',
       'school_district_Enumclaw_School_District',
       'school_district_Federal_Way_School_District',
       'school_district_Highline_School_District',
       'school_district_Issaquah_School_District',
       'school_district_Kent_School_District',
       'school_district_Lake_Washington_School_District',
       'school_district_Mercer_Island_School_District',
       'school_district_Northshore_School_District',
       'school_district_Renton_School_District',
       'school_district_Riverview_School_District',
       'school_district_Seattle_Public_Schools',
       'school_district_Shoreline_S

In [176]:
X = df.drop("price", axis=1)
y = df[["price"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=6)

In [177]:
model = LGBMRegressor()



model.fit(X_train, y_train)

predictions = model.predict(X_test)


# Calculate RMSE
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, predictions)

print(f"\n\nRMSE: {rmse}")
print(f"\n\nR^2: {r2}")



RMSE: 105917.03989219587


R^2: 0.920836405700898


In [178]:
mini = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
)


mini = pd.merge(mini, crime_rate, on="zipcode", how="left")
mini

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,property_crime_rate_closest_city,violent_crime_rate_closest_city,school_district
0,930000305,20141110T000000,4,1.75,2120,7680,1.0,0,0,4,7,1060,1060,1950,0,98177,47.7172,-122.361,1530,7680,2418.7,132.9,Shoreline School District
1,9541600015,20150211T000000,4,2.25,2010,15375,1.0,0,0,4,8,2010,0,1957,0,98005,47.5956,-122.174,1930,15375,3126.4,109.7,Bellevue School District
2,7338000150,20150129T000000,2,1.00,1070,4200,1.0,0,0,4,6,1070,0,1983,0,98002,47.3336,-122.215,1150,4200,5435.5,408.1,Auburn School District
3,6113400046,20140723T000000,4,2.50,1890,15770,2.0,0,0,4,7,1890,0,1968,0,98166,47.4281,-122.343,2410,15256,4727.7,392.5,Highline School District
4,291310170,20140804T000000,3,2.50,1600,2610,2.0,0,0,3,8,1600,0,2005,0,98027,47.5344,-122.068,1445,1288,3091.5,28.6,Issaquah School District
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,8902000267,20150402T000000,4,2.75,2260,7209,1.0,0,3,3,7,1330,930,2002,0,98125,47.7088,-122.302,1790,10860,2221.1,60.0,Seattle Public Schools
77,7856550240,20140710T000000,5,2.25,3480,9200,2.0,0,0,3,8,3480,0,1979,0,98006,47.5585,-122.153,3130,9200,2321.9,17.5,Bellevue School District
78,7923500060,20140922T000000,5,2.75,2580,9242,2.0,0,2,4,8,1720,860,1967,0,98007,47.5943,-122.133,2240,9316,3126.4,109.7,Bellevue School District
79,8898700880,20150317T000000,2,2.00,1590,8000,1.0,0,0,3,7,910,680,1984,0,98055,47.4590,-122.205,1590,8364,5794.1,300.0,Renton School District


In [179]:
mini = data_transform(mini)

# missing_columns = [
#     "school_district_Enumclaw_School_District",
#     "school_district_Tahoma_School_District",
#     "school_district_Tukwila_School_District",
#     "school_district_Vashon_Island_School_District",
# ]

# for column in missing_columns:
#     mini[column] = 0

In [180]:
predictions = model.predict(mini)
df = pd.DataFrame(predictions)
df.to_csv("team3-module3-predictions.csv", index=False, header=["price"])